In [1]:
# 运行环境： python-3.6.12 tensorflow-1.14.0  
import numpy as np
import pandas as pd
import tensorflow as tf
from geopy.distance import geodesic
import scipy.stats
import os

/home/zzy/anaconda3/envs/traffic/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zzy/anaconda3/envs/traffic/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zzy/anaconda3/envs/traffic/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zzy/anaconda3/envs/traffic/lib/python3.6/site-

In [1]:
import sys
sys.executable

'/home/zzy/anaconda3/bin/python'

In [2]:
tf.__version__

'1.14.0'

In [3]:
# 导入数据：Dataset：
SIP_matrixS = np.load('Matrix_Aff_Static.npy') 
SIP_matrixS = np.where(SIP_matrixS>1, 0.5,SIP_matrixS)
SIP_E = np.load('SIP_Time5_Weather10-Interval30-201701_03.npy') 
GPS_Grid_SIP = pd.read_csv('GPS_Grid_SIP-1.csv')
SIP_30min_flow = np.load('SIP_30m_Flow.npy')

# Settings:
Inte_Min = 30
DayIntr = np.int(1440/Inte_Min)
Perd = 6
D = 1
C = 3
Node = 108
week = [7,14]
Ex_num = SIP_E.shape[1]

hidden_dim_F = 72
# num of input signals
input_dim_F = 108
# num of output signals
output_dim_F = 108

Perd_no = 5


 ## Network Parameters
# length of input signals
output_seq_len = 1 
# num of stacked lstm layers 
num_stacked_layers = 2
# gradient clipping - to avoid gradient exploding
GRADIENT_CLIPPING = 2.5 
# input size 表示输入序列的长度，比如输入序列长度是20，输出12，就是利用前20个连续序列来预测第21-32个值 
# size of LSTM Cell
hidden_dim = 32 
# num of input signals
input_dim = Node
# num of output signals
output_dim = Node
kernel_Rep = 10

Stacklayer = 2
Hidden_Dim = 100

In [4]:
## 计算距离：
Matrix_dist = np.zeros([Node,Node])
Matrix_dist1 = np.zeros([Node,Node])
for i in range(len(GPS_Grid_SIP)):
    for j in range(len(GPS_Grid_SIP)):
        Matrix_dist[i,j] = geodesic((GPS_Grid_SIP['lat'][i],GPS_Grid_SIP['long'][i]),(GPS_Grid_SIP['lat'][j],GPS_Grid_SIP['long'][j])).km
Matrix_dist1 = np.exp(-Matrix_dist/8)
def Adjacent_Matrix(Flow1,Average = False):
    if Flow1.shape[0] == 1:
        Flow1 = np.squeeze(Flow1)
    (Node,Time_Inte) = Flow1.shape
   # print(Flow1.shape)
    a1 = 0.5
    a2 = 0.5
    matrix = np.zeros([Time_Inte,Node,Node])
    for k in range(Time_Inte):
        for i in range(Node):
            for j in range(Node):
                if i > j:        
                    prox2 = (Flow1[i,k]*Flow1[j,k]/Matrix_dist[i,j]) #算gravity
                    prox2 = np.log(prox2+1)/10 
                    matrix[k,i,j] = a1* Matrix_dist1[i,j] + a2 * prox2                    
    matrix1 = matrix + np.transpose(matrix,[0,2,1])
    if Average == True:
        matrix = np.mean(matrix,axis = 0)
        matrix = np.expand_dims(matrix,axis = 0)
    return matrix

In [5]:
def ST_Variance(Input_Batch,Matrix_Aff):
    Var_Perd = np.std(Input_Batch[:,:,0], axis = 0)
    Var_Close = []
    for i in range(Perd_no):
        var1 = np.std(Input_Batch[i,:,:], axis = 1)
        Var_Close.append(var1)
    Var_Close = np.array(Var_Close)
    Var_Spat = []
    Avera_FlowP = np.mean(Input_Batch,axis = 2)
    for i in range(Perd_no):
        Neighb = Matrix_Aff[0,i]
        Neighb = np.where(Neighb>1.05,1,0)+(np.identity(Node))
        Avera_FlowP1 = Avera_FlowP[i,:].reshape([1,Node])
        # Input_Batch[0,i,:,5]
        Avera_FlowP1 = np.tile(Avera_FlowP1,(Node,1)) 
        NeighborValue = np.multiply(Avera_FlowP1,Neighb)
        Vara = np.zeros([Node,1])
        for k in range(Node):
            Vara[k] = np.std(np.unique(NeighborValue[k]))
        Var_Spat.append(Vara)
    Var_Spat = np.squeeze(np.array(Var_Spat),axis = 2)
    
    #print(Var_Spat.shape,Var_Close.shape,Var_Close.shape)
    ST_Var = 0.333* Var_Close + 0.333 * Var_Perd + 0.333 * Var_Spat
    return Var_Close, Var_Perd, Var_Spat, ST_Var

In [6]:
def Data_OrganSample(t,Corup = False, Degree = 1):
    # t = 14*48 + 20
    # Stat_ID = 10
    Y = SIP_30min_flow[:,t]
    # 近1 period closeness:
    Pc = SIP_30min_flow[:,t-Perd:t]
    #--------- Generate  Noise ---------#
    Add_Noise = np.random.randn(Perd_no,Node,Perd)*25*Degree
    Qua_Noise = np.mean(np.abs(Add_Noise),axis = -1)
    
    #-----------------------------------#
    Pc = Pc.reshape([1,Node,Perd])
    SIP_Ec = np.mean(SIP_E[t-Perd:t],axis = 0)
    SIP_Ec = np.expand_dims(SIP_Ec,axis = 0)
    # 近3天 对应period
    Pd = []
    SIP_Ed = []
    for i in range(C,0,-1):
        Pd.append(SIP_30min_flow[:,t-(DayIntr*i+Perd):t-DayIntr*i])   #3,2,1
        SIP_Ed_temp = np.mean(SIP_E[t-(DayIntr*i+Perd):t-DayIntr*i],axis = 0)
        SIP_Ed.append(SIP_Ed_temp)
    Pd = np.array(Pd)
    SIP_Ed = np.array(SIP_Ed)    

    # 近往前一周 对应period的summarize:
    WeekS = []
    SIP_EW = []
    for i in range(week[1],week[0],-1):
        if t-(DayIntr*i+Perd)>0:
            WeekS.append(SIP_30min_flow[:,t-(DayIntr*i+Perd):t-DayIntr*i])
            SIP_EW_temp = np.mean(SIP_E[t-(DayIntr*i+Perd):t-DayIntr*i],axis = 0)
            SIP_EW.append(SIP_EW_temp)
    WeekS = np.array(WeekS)
    WeekS1 = np.mean(WeekS,axis = 0)
    WeekS1 = WeekS1.reshape([1,Node,Perd])
    SIP_EW = np.mean(SIP_EW,axis = 0)
    SIP_EW = np.expand_dims(SIP_EW,axis = 0)

    FlowData = np.concatenate([WeekS1,Pd,Pc],axis = 0)
    Exte = np.concatenate([SIP_EW,SIP_Ed,SIP_Ec],axis = 0)
    #print(Pc.shape)
    #------------------Add Noise------------------#
    if Corup == True:
        FlowData = FlowData + Add_Noise
    # Adjacent Matrix
    Matrix_c = Adjacent_Matrix(Pc,Average = True)
    Pd_A = np.mean(Pd,axis = 2)
    Pd_A = np.expand_dims(Pd_A,axis = -1)
    Pd_A = Pd_A.transpose()
    Matrix_d = Adjacent_Matrix(Pd_A)
    Matrix_w = Adjacent_Matrix(WeekS1,Average = True)
    #print("Inputshape;",Pc.shape, Pd_A.shape,WeekS1.shape)
    #print(Matrix_c.shape,Matrix_d.shape,Matrix_w.shape)
    Matrix_Aff = np.concatenate((Matrix_c,Matrix_d,Matrix_w),axis=0)
    
    return FlowData, Exte, Y, Matrix_Aff, Qua_Noise

In [7]:
# JS 散度
def JS_divergence(p,q):
    M=(p+q)/2
    return 0.5*scipy.stats.entropy(p, M)+0.5*scipy.stats.entropy(q, M)

In [8]:
tt = np.arange(768,816)
tt

array([768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780,
       781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793,
       794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806,
       807, 808, 809, 810, 811, 812, 813, 814, 815])

In [9]:
def generate_samples(batch_size, corup = False,degree = 0):
    tt = np.random.choice(range(14*48,4320), batch_size)
  #  tt = np.arange(768,816)
    Input_Batch = []
    Input_Ext_B = []
    Matrix_Aff_B = []
    Output_Y_B = []
    Qua_Noise_B = []
    ST_VAR_B = []
    for i in tt:
        FlowData, Exte, Y, Matrix_Aff, Qua_Noise = Data_OrganSample(i,Corup = corup, Degree = degree)
        Input_Batch.append(FlowData)
        Input_Ext_B.append(Exte)
        Matrix_Aff_B.append(Matrix_Aff)
        Output_Y_B.append(Y)
        Qua_Noise_B.append(Qua_Noise)
        _,_,_, ST_var = ST_Variance(FlowData,Matrix_Aff)
        ST_VAR_B.append(ST_var)
    Input_Batch = np.array(Input_Batch)/100
    Input_Ext_B = np.array(Input_Ext_B)
    Matrix_Aff_B = np.array(Matrix_Aff_B)
    Output_Y_B = np.array(Output_Y_B) /100
    Qua_Noise_B = np.array(np.array(Qua_Noise_B))/100
    Output_Y_B = np.expand_dims(Output_Y_B, axis = 1)
    # 除以10 作为指示
    ST_VAR_B = np.array(ST_VAR_B)/100  #00
    print(str(batch_size)+"sample generating fisnished!")
    return Input_Batch,Input_Ext_B,Matrix_Aff_B,Output_Y_B,Qua_Noise_B,ST_VAR_B

In [10]:
Input_Batch,Input_Ext_B,Matrix_Aff_B,Output_Y_B, Qua_Noise_B, ST_VAR_B = generate_samples(1, corup = True,degree = 1)

1sample generating fisnished!


In [11]:
Input_Batch.shape

(1, 5, 108, 6)

In [13]:
Output_Y_B_sm = np.mean(Output_Y_B, axis = 0)
np.argsort(-Output_Y_B_sm[0])

array([  0,   2,   4,  36,  29,  32,   3,   1,  13,  37,   5,  44,  43,
        38,  48,  15,  10,  76,  21,  64,   6,   9,  30,  53,  12,  16,
        56,  31,  50,  33,  67,  46,  47,  58,  78,  39,  61,   7,  57,
         8,  59,  35,  73,  27,  34,  17,  41,  45,  63,  42,  19,  49,
        25,  72,  66,  23,  26,  79,  40,  28,  80,  14,  71,  22,  74,
        84,  51,  86,  62,  69,  70,  68,  20,  83,  75,  94,  90,  54,
        60,  85,  65,  82,  18,  11,  93,  55,  87,  96,  88,  24,  77,
        52,  81,  95, 100, 103,  91,  92, 102,  98,  89,  97,  99, 104,
       105, 101, 107, 106])

In [123]:
def build_graph(feed_previous = False,reuse_variables=False):
    #tf.enable_eager_execution()
    tf.reset_default_graph()
    global_step = tf.Variable(
                  initial_value=0,
                  name="global_step",
                  trainable=False,
                  collections=[tf.GraphKeys.GLOBAL_STEP, tf.GraphKeys.GLOBAL_VARIABLES])

    def gen_gcn_weights():
        gcn_weights = {}
        N_num=32
        Attribute_Num = 26 
        
        GCN_Pred = 32
        Attr_Num_Pre = 6
        Attr_Num_Pre01 = 1
        GCN_Pred01 = 6
        hidden_dim_F = 64
        # num of input signals
        input_dim_F = 108
        # num of output signals
        output_dim_F = 108
        gcn_weights['FMGCN1'] = tf.Variable(tf.truncated_normal(shape=(Attribute_Num, N_num), mean=0, stddev=0.1))
        gcn_weights['FMGCN'] = tf.Variable(tf.truncated_normal(shape=(N_num, N_num), mean=0, stddev=0.1))
        gcn_weights['FMGCN_N'] = tf.Variable(tf.truncated_normal(shape=(N_num, 1), mean=0, stddev=0.1))
        gcn_weights['PredGCN1'] = tf.Variable(tf.truncated_normal(shape=(Attr_Num_Pre, GCN_Pred), mean=0, stddev=0.1))
        gcn_weights['PredGCN'] = tf.Variable(tf.truncated_normal(shape=(GCN_Pred, GCN_Pred), mean=0, stddev=0.1))
        gcn_weights['PredGCN_N'] = tf.Variable(tf.truncated_normal(shape=(GCN_Pred, 1), mean=0, stddev=0.1))
        gcn_weights['PredGCN_01'] = tf.Variable(tf.truncated_normal(shape=(Attr_Num_Pre01, GCN_Pred01), mean=0, stddev=0.1))
        gcn_weights['PredGCN_011'] = tf.Variable(tf.truncated_normal(shape=(GCN_Pred01, GCN_Pred01), mean=0, stddev=0.1))
        gcn_weights['PredGCN_N01'] = tf.Variable(tf.truncated_normal(shape=(GCN_Pred01, 1), mean=0, stddev=0.1))
        gcn_weights['Seq_out_WF'] = tf.Variable(tf.truncated_normal(shape=(hidden_dim_F, output_dim_F), dtype = tf.float32, mean=0, stddev=0.1))
        gcn_weights['Seq_out_bF'] = tf.Variable(tf.truncated_normal(shape=(output_dim_F,), dtype = tf.float32,mean=0, stddev=0.1))
        return gcn_weights

    weights=gen_gcn_weights()
    def weight_variable(shape):
        initial=tf.truncated_normal(shape,stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial=tf.truncated_normal(shape,stddev=0.1)
        return tf.Variable(initial)

    #Data Quality evaluation:
    def get_cos_distance(X1, X2):
        return tf.reduce_mean(tf.multiply(X1,X2),axis = 2)/(tf.sqrt(tf.reduce_sum(tf.square(X2), axis=2))*tf.sqrt(tf.reduce_sum(tf.square(X2), axis=2)))

    def Uncertain_T(Input_Seq):   
        Repstore = []
        for i in range(Perd_no):
            Rep_w = weight_variable([Perd,kernel_Rep])  
            Rep_Per_b = bias_variable([kernel_Rep])
            Rep1 = tf.nn.relu(tf.matmul(Input_Seq[:,i,:,:],Rep_w) + Rep_Per_b, name='Rep_'+str(i))
            Rep_ex = weight_variable([Ex_num,kernel_Rep])  
            Rep_ex_b = bias_variable([kernel_Rep])
            Ex_Rep = tf.nn.relu(tf.matmul(Input_Ext[:,i,:],Rep_ex) + Rep_ex_b, name='Rep2'+str(i))
            Ex_Rep = tf.tile(tf.expand_dims(Ex_Rep, axis = 1),[1,Node,1])
            Rep = tf.add(Rep1, Ex_Rep)       
            Repstore.append(Rep)
        Repstore_Tensor = tf.convert_to_tensor(Repstore)
        Repstore_Tensor = tf.transpose(Repstore_Tensor, [1, 0, 2, 3]) 
        tf.multiply(Repstore_Tensor[:,0,:,:],Repstore_Tensor[:,1,:,:])
        Sim_S = []
        # 两两计算相似度
        for i in range(Perd_no):
            Sim = []
            for j in range(Perd_no):
                if i!= j:
                    Sim.append(get_cos_distance(Repstore_Tensor[:,i,:,:],Repstore_Tensor[:,j,:,:]))
            Sim = tf.convert_to_tensor(Sim)
            Sim = tf.reduce_mean(Sim,axis = 0)
            Sim_S.append(Sim)
        Sim_S =  tf.convert_to_tensor(Sim_S)  
        print(Sim_S.shape)
        Sim_S = tf.transpose(Sim_S,[1,0,2])
        Sim_2_Uncer = tf.exp(-Sim_S)
        # Adjust for uncertainty-tensor
        Uncer_Tw = weight_variable([Node,Node])  
        Uncer_Tb = weight_variable([Node])  
        Uncer_T = tf.nn.relu(tf.matmul(Sim_2_Uncer,Uncer_Tw) + Uncer_Tb)
        return Uncer_T



    def Long_termGCN(input1, input_external, Adj_M):
        N_num = 32
        ex_num = input_external.shape[-1]  
        layer1_temp = tf.matmul(Adj_M, input1)
        layer_1_output = tf.map_fn(lambda x: tf.matmul(x, weights['PredGCN1']), layer1_temp)
        # print(layer_1_output.shape)
        #定义第二层网络结构 GCN
        #layer2_temp = tf.map_fn(lambda x: tf.matmul(Adj_M, x), layer_1_output)
        layer2_temp = tf.matmul(Adj_M, layer_1_output)
        layer_2_output = tf.map_fn(lambda x: tf.matmul(x, weights['PredGCN_N']), layer2_temp) ##
        layer_2_output=tf.nn.leaky_relu(layer_2_output,alpha = 0.8)    
        img_shape = [4,Node,64]
        #Wx_plus_b = tf.Variable(tf.random_normal(img_shape))
        axis = list(range(len(img_shape) - 1))
        wb_mean, wb_var = tf.nn.moments(layer_2_output, [0,1])
        scale = tf.Variable(tf.ones([1]))
        offset = tf.Variable(tf.zeros([1]))
        variance_epsilon = 0.001
        layer_2_output = tf.nn.batch_normalization(layer_2_output, wb_mean, wb_var, offset, scale, variance_epsilon)

        layer_4_output = tf.nn.relu(layer_2_output)        

        output= tf.squeeze(layer_4_output , -1) 
        keep_prob=0.5
        External_W = weight_variable([Ex_num,Node])  
        External_b = bias_variable([Node])
        External_out = tf.matmul(input_external,External_W) + External_b
        Output = External_out + output
        return Output


    def Short_termGCN(input1, input_external, Adj_M):
        N_num = 6
        input1 = tf.expand_dims(input1, axis = -1)
        ex_num = input_external.shape[-1]  
        layer1_temp = tf.matmul(Adj_M, input1)
        layer_1_output = tf.map_fn(lambda x: tf.matmul(x, weights['PredGCN_01']), layer1_temp)
        # print(layer_1_output.shape)
        #定义第二层网络结构 GCN
        #layer2_temp = tf.map_fn(lambda x: tf.matmul(Adj_M, x), layer_1_output)
        layer2_temp = tf.matmul(Adj_M, layer_1_output)
        layer_2_output = tf.map_fn(lambda x: tf.matmul(x, weights['PredGCN_N01']), layer2_temp) ##
        layer_2_output=tf.nn.leaky_relu(layer_2_output,alpha = 0.8) 
        img_shape = [4,Node,64]
        #Wx_plus_b = tf.Variable(tf.random_normal(img_shape))
        axis = list(range(len(img_shape) - 1))
        wb_mean, wb_var = tf.nn.moments(layer_2_output, [0,1])
        scale = tf.Variable(tf.ones([1]))
        offset = tf.Variable(tf.zeros([1]))
#         scale = tf.Variable(tf.ones([N_num]))
#         offset = tf.Variable(tf.zeros([N_num]))
        variance_epsilon = 0.001
        layer_2_output = tf.nn.batch_normalization(layer_2_output, wb_mean, wb_var, offset, scale, variance_epsilon)
        layer_4_output = tf.nn.relu(layer_2_output)         

        output= tf.squeeze(layer_4_output , -1) 
        keep_prob = 0.5
        External_W = weight_variable([Ex_num,Node])  
        External_b = bias_variable([Node])
     #   External_out = tf.nn.relu(tf.matmul(input_external,External_W) + External_b)
        External_out = tf.matmul(input_external,External_W) + External_b
        Output = External_out + output
        return Output


    #——————————————————LSTM——————————————————#
    def lstm(Output6,Output_Seq,num_stacked_layers,input_size, hidden_dim): 
        weightslstm={
         'in':tf.Variable(tf.random_normal(shape=[input_size,hidden_dim], mean=0, stddev=0.1)), 
         'out':tf.Variable(tf.random_normal(shape=[hidden_dim,input_size], mean=0, stddev=0.1)) 
        }
        biaseslstm={
                'in':tf.Variable(tf.random_normal(shape=[hidden_dim,], mean=0, stddev=0.1)), 
                'out':tf.Variable(tf.random_normal(shape=[input_size,], mean=0, stddev=0.1)) 
               }


        ## Seq2Seq Parameters
        ## Parameters
        enc_inp = Output6

        print("len_enc_inp:",len(enc_inp))
        print(enc_inp)
        # Decoder: target outputs
        target_seq = [
            tf.placeholder(tf.float32, shape=(None, input_size), name="y".format(t))
              for t in range(output_seq_len)
        ]

        target_seq = Output_Seq
        print("len_target_seq:",len(target_seq))
        dec_inp = [ tf.zeros_like(target_seq[0], dtype=tf.float32, name="GO") ] + target_seq[:-1]
    
    
        cells = []
        for i in range(num_stacked_layers):
            with tf.variable_scope('RNN_{}'.format(i)):
                cells.append(tf.contrib.rnn.LSTMCell(num_units = hidden_dim, activation = tf.nn.leaky_relu)) # activation = tf.nn.leaky_relu

        cell = tf.contrib.rnn.MultiRNNCell(cells)
        def _rnn_decoder(decoder_inputs,
                        initial_state,
                        cell,
                        loop_function=None,
                        scope=None):

            state = initial_state
            outputs = []
            prev = None
            for i, inp in enumerate(decoder_inputs):
              if loop_function is not None and prev is not None:
                with tf.variable_scope("loop_function", reuse=True):
                  inp = loop_function(prev, i)
              if i > 0:
                tf.get_variable_scope().reuse_variables()
              output, state = cell(inp, state)
              outputs.append(output)
              if loop_function is not None:
                prev = output
            return outputs, state


        def _basic_rnn_seq2seq(encoder_inputs,
                              decoder_inputs,
                              cell,
                              feed_previous,
                              dtype=tf.float32,
                              scope=None):
        #  """Basic RNN sequence-to-sequence model.
        #  """
        # Encoder  
        # 运行 encoder,把待编码的输入RNN中得到enc_state
            enc_cell = cell
            _, enc_state = tf.contrib.rnn.static_rnn(enc_cell, encoder_inputs, dtype=dtype)
            # Decoder 
            #如何feed_previous = True 的话 就把先前decoder 输出的放到下一个时间步去进行decoder
            if feed_previous:
                return _rnn_decoder(decoder_inputs, enc_state, cell, _loop_function)
            else:
                return _rnn_decoder(decoder_inputs, enc_state, cell)
        #print(tf.get_variable_scope().reuse)
        def _loop_function(prev, _):
          '''Naive implementation of loop function for _rnn_decoder. Transform prev from 
          dimension [batch_size x hidden_dim] to [batch_size x output_dim], which will be
          used as decoder input of next time step '''
          return tf.matmul(prev, weightslstm['out']) + biaseslstm['out']
        
        dec_outputs, dec_memory = _basic_rnn_seq2seq(
            enc_inp, 
            dec_inp,  ##输入带有context的数据
            cell, 
            feed_previous = feed_previous
        )
        Pred = [tf.nn.relu(tf.matmul(i, weightslstm['out'])+biaseslstm['out']) for i in dec_outputs]
        return Pred



    def FM_GCN_UST(Input_Ext1, Matrix_1):
        Time_ExN = 5
        Weath_ExN = 10
        Embed_N = 10
        Intract_No = 6
        N_num = 32
        Input_Ext2 = tf.expand_dims(Input_Ext1, axis = 1)
        Input_Ext2 = tf.tile(Input_Ext2,[1,108,1]) 
        Input_Ext2.shape
        Time_Ex = Input_Ext2[:,:, 0:Time_ExN]
        Weath_Ex = Input_Ext2[:,:,Time_ExN : Time_ExN + Weath_ExN]

        Time_E_w = weight_variable([Time_ExN,Embed_N])  
        Time_E_b = bias_variable([Embed_N])
    #    Time_E = tf.nn.leaky_relu(tf.matmul(Time_Ex,Time_E_w)+Time_E_b, alpha = 0.8)
        Time_E = tf.matmul(Time_Ex,Time_E_w)+Time_E_b
        
        Weath_E_w = weight_variable([Weath_ExN,Embed_N])  
        Weath_E_b = bias_variable([Embed_N])
       # Weath_E = tf.nn.leaky_relu(tf.matmul(Weath_Ex,Weath_E_w)+Weath_E_b, alpha = 0.8)
        Weath_E = tf.matmul(Weath_Ex,Weath_E_w)+Weath_E_b

        InteAct_w = weight_variable([Embed_N,Intract_No]) 
        InteAct_b = bias_variable([Intract_No])
        Inter_R = tf.nn.leaky_relu(tf.matmul(tf.multiply(Time_E,Weath_E),InteAct_w)+InteAct_b, alpha = 0.8)
        Graph_R = tf.concat([Time_E,Weath_E,Inter_R],axis = 2)


        #定义第一层网络结构 GCN
        layer1_temp = tf.matmul(Matrix_1, Graph_R)
        layer_1_output = tf.map_fn(lambda x: tf.matmul(x, weights['FMGCN1']), layer1_temp)
        # print(layer_1_output.shape)
        #定义第二层网络结构 GCN
        layer2_temp = tf.matmul(Matrix_1, layer_1_output)
        layer_2_output = tf.map_fn(lambda x: tf.matmul(x, weights['FMGCN']), layer2_temp)
        layer_2_output=tf.nn.leaky_relu(layer_2_output,alpha = 0.8)    
        img_shape = [4,Node,64]
        #Wx_plus_b = tf.Variable(tf.random_normal(img_shape))
        axis = list(range(len(img_shape) - 1))
        wb_mean, wb_var = tf.nn.moments(layer_2_output, [0,1])
        scale = tf.Variable(tf.ones([N_num]))
        offset = tf.Variable(tf.zeros([N_num]))
        variance_epsilon = 0.001
        layer_2_output = tf.nn.batch_normalization(layer_2_output, wb_mean, wb_var, offset, scale, variance_epsilon)

        #定义第三层网络结构 GCN
        #layer3_temp = tf.map_fn(lambda x: tf.matmul(Adj_M, x), layer_2_output)
        layer3_temp = tf.matmul(Matrix_1, layer_2_output)
        layer_3_output = tf.map_fn(lambda x: tf.matmul(x, weights['FMGCN']), layer3_temp)

        #定义第四层网络结构 GCN
        #layer4_temp = tf.map_fn(lambda x: tf.matmul(Adj_M, x), layer_3_output)
        layer4_temp = tf.matmul(Matrix_1, layer_3_output)
        layer_4_output = tf.map_fn(lambda x: tf.matmul(x, weights['FMGCN_N']), layer4_temp)      
        layer_4_output = tf.nn.leaky_relu(layer_4_output, alpha = 0.8)

        wb_mean, wb_var = tf.nn.moments(layer_4_output, [0,1])
        scale = tf.Variable(tf.ones([1]))
        offset = tf.Variable(tf.zeros([1]))
        layer_4_output = tf.nn.batch_normalization(layer_4_output, wb_mean, wb_var, offset, scale, variance_epsilon)
        layer_4_output = tf.nn.leaky_relu(layer_4_output, alpha = 0.8)             
        output= tf.squeeze(layer_4_output , -1) 
        return output

    # mobility-uncertainty re-calibrate gate 门机制
    def MuRG(Pred, Uncert):
        print("1",Pred.shape, Uncert.shape)
        #Pred = Output6[0,:,:]
        #Uncert = Output6[1,:,:]
        Gate_Feat = tf.concat([Pred, Uncert],axis = -1)
        Gate_w = weight_variable([Node*2,Node])  
        Gate_b = bias_variable([Node])
        Gate_V = tf.nn.tanh(tf.matmul(Gate_Feat,Gate_w)+Gate_b)
        Bias = tf.multiply(Gate_V,Uncert)
        Pred_Up = tf.add(Pred, Bias) 
        Uncert_Up = tf.subtract(Uncert, Bias)
        return Pred_Up, Uncert_Up

    #-----------输入input———————————————#
    # 5 (Attribute)    //      8 interval = 2 (expected) + 6(continous)
    # Acc_Risk, Pick, Speed, Pick_diff, Speed_diff
    Input_Seq = tf.placeholder(shape=(None, Perd_no, Node, Perd),dtype=tf.float32) 
    Input_Ext = tf.placeholder(shape=(None, Perd_no, Ex_num),dtype=tf.float32)
    Matrix_Aff = tf.placeholder(shape=(None, Perd_no, Node, Node),dtype=tf.float32)
    Output_Seq = tf.placeholder(shape=(None, output_seq_len, Node),dtype=tf.float32)
    Data_quality = tf.placeholder(shape=(None, Perd_no, Node),dtype=tf.float32)
    ST_variance = tf.placeholder(shape=(None, Perd_no, Node),dtype=tf.float32)
    # Repstore = tf.placeholder(shape=(None, Perd_no, Node, kernel_Rep),dtype=tf.float32)

    # 调用第一类internal data-quality based uncertainty evaluation
    Uncer_T = Uncertain_T(Input_Seq)
 
    print("Uncer_T:", Uncer_T.shape)


    # 预测的时间段 分别是4个中长期，6个最近的短期 Prediction for the next-interval
    Output6 = []
    for i in range(1,4):
        Output6.append(Long_termGCN(Input_Seq[:,i,:,:], Input_Ext[:,i,:],Matrix_Aff[:,i,:,:]))
    for j in range(6):
        Output6.append(Short_termGCN(Input_Seq[:,4,:,j], Input_Ext[:,4,:],Matrix_Aff[:,4,:,:]))
    Output_Seq1 = tf.squeeze(Output_Seq, axis = 1)
    Output_seq = []
    for j in range(output_seq_len):
        Output_seq.append(Output_Seq1)
    # Output6 = tf.convert_to_tensor(Output6)
    # Output6 = tf.transpose(Output6, [1, 0, 2]) 
   # print("Output6:",Output6.shape)
    with tf.variable_scope("Pred_RNN"):  
        Pred_Result = lstm(Output6,Output_seq,Stacklayer,Node,Hidden_Dim)

    # 调用第二类external uncertainty FM-GCN网络
    Output_UnST = []
    for i in range(5):
        Output_UnST.append(FM_GCN_UST(Input_Ext[:,i,:],Matrix_Aff[:,i,:,:]))
    Output_UnST = tf.convert_to_tensor(Output_UnST)
    print(Output_UnST.shape)
    Output_UnST = tf.transpose(Output_UnST,[1,0,2])
    print("Output_UnST:",Output_UnST.shape)

    # 对两次求出来的Uncertainty进行一次聚合
    Uncertain_WT = weight_variable([Perd_no, Node])
    Uncertain_WST = weight_variable([Perd_no, Node])
    # Unc_Fuse = tf.multiply(Uncer_T, Uncertain_WT) + tf.multiply(Output_UnST,Uncertain_WST)
    Unc_Fuse = Uncer_T + Output_UnST
    Unc_Fuse_L = []
    for i in range(Perd_no):
        Unc_Fuse_L.append(Unc_Fuse[:,i,:])
    # 对聚合的uncertainty运用LSTM 捕获时间关联
    Output_Unc_Fuse_L = []
    for j in range(output_seq_len):
        Output_Unc_Fuse_L.append(ST_variance[:,-1,:])
    with tf.variable_scope("Uncert_RNN"):  
        Uncert_Result = lstm(Unc_Fuse_L,Output_Unc_Fuse_L,Stacklayer,Node,Hidden_Dim)
    Uncert_Result = tf.convert_to_tensor(Uncert_Result)
    print("Uncert_Result_shape", Uncert_Result.shape)
  #  print("Input_MuRG",Pred_Result.shape,Uncert_Result.shape)
    Pred_Result = tf.convert_to_tensor(Pred_Result)
    Pred_Result = tf.squeeze(Pred_Result, axis = 0)
    Uncert_Result = tf.squeeze(Uncert_Result, axis = 0)
    # 输出最终的预测结果
    Pred_Up, Uncert_Up = MuRG(Pred_Result, Uncert_Result)
    print("Pred_shape, Uncert_Up:",Pred_Up.shape,Uncert_Up.shape)
    reg_loss = 0
    output_lossP = 0
    output_lossUT = 0
    output_lossUST = 0
    
    ST_variance1 = ST_variance[:,-1,:]
    # Loss function
    
    
    print("####################")
    print(Pred_Result.shape, Output_Seq1.shape)
    print(Pred_Up.shape, Output_Seq1.shape)
    print(ST_variance1.shape, Uncert_Up.shape)    
    print("####################")
    
    # Loss function 
    output_lossP = tf.reduce_mean(tf.pow(Pred_Up - Output_Seq1, 2)) + 2 * tf.reduce_mean(tf.div(tf.pow(Pred_Up - Output_Seq1, 2), Output_Seq1+0.1))
    #1.2* tf.reduce_mean(tf.pow(Pred_Up - Output_Seq1, 2)) +  # + tf.reduce_mean(tf.pow(Pred_Result - Output_Seq1, 2)) # 两次预测 一次直接预测 还有一次re-calibrate的
#     for _y, _Y in zip(Pred_Up, Output_Seq1):
#        output_lossP += tf.reduce_mean(tf.pow(_y - _Y, 2))
#    output_lossU = tf.reduce_mean(tf.pow(Uncert_Up - Uncert_Noise, 2))
 #   output_lossUT = tf.reduce_mean(tf.pow(Data_quality - , 2)) tf.reduce_mean(tf.square(ST_variance1 - Uncert_Result)) +
    output_lossUST = tf.reduce_mean(tf.square(ST_variance - Output_UnST))  + tf.reduce_mean(tf.square(ST_variance1 - Uncert_Result)) #最后的时空方差
    
    for tf_var in tf.trainable_variables():
        if 'Seq_out_' in tf_var.name in tf_var.name:
            reg_loss += tf.reduce_mean(tf.nn.l2_loss(tf_var))
             
            
    lambda_l2_reg = 0.0001
    learning_rate = tf.train.exponential_decay(1e-3,global_step,decay_steps=10,decay_rate=0.98,staircase=True)  
    loss = output_lossP +  output_lossUST + tf.norm(Uncert_Up, ord=2) + lambda_l2_reg * reg_loss  # tf.norm表示本身波动 uncertainty应该较小
    
    # Convert Output6 to tensor
    Output6 = tf.convert_to_tensor(Output6)
    Output6 = tf.transpose(Output6, [1, 0, 2]) 


    with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
        optimizer = tf.contrib.layers.optimize_loss(
        loss=loss,
        learning_rate=learning_rate,
        global_step=global_step,
        optimizer='Adam',
        clip_gradients=GRADIENT_CLIPPING)
    saver = tf.train.Saver

    return dict(
        Input_Seq = Input_Seq,
        Output_Seq = Output_Seq,
        Input_Ext = Input_Ext, 
        Matrix_Aff = Matrix_Aff,
        Uncer_T = Uncer_T,
        Uncert_Result = Uncert_Result,
        Output_UnST = Output_UnST,
        Data_quality = Data_quality,
        ST_variance = ST_variance,
        train_op = optimizer, 
        loss=loss,
        saver = saver, 
        Output6 = Output6,
        Pred_Result = Pred_Result,
        Pred_Up = Pred_Up,
        learning_rate = learning_rate,
        Uncert_Up = Uncert_Up
        )

In [92]:
ST_VAR_B[10,-1,:]

array([0.24335656, 0.2223248 , 0.17582949, 0.31906779, 0.24747572,
       0.23154602, 0.30173432, 0.26573328, 0.34397442, 0.42200978,
       0.24561142, 0.35061831, 0.3079706 , 0.48431429, 0.2347914 ,
       0.2137994 , 0.36124648, 0.41501386, 0.31506554, 0.47889774,
       0.24962062, 0.19052253, 0.18912164, 0.20514922, 0.38026633,
       0.59334193, 0.27525122, 0.14088961, 0.29202968, 0.25868568,
       0.36726125, 0.14372388, 0.18430205, 0.20159876, 0.24683507,
       0.19380061, 0.19529018, 0.24634292, 0.27997093, 0.29997253,
       0.16904874, 0.25367248, 0.35731018, 0.2225775 , 0.50894997,
       0.30476888, 0.40007775, 0.19223931, 0.28377342, 0.45358848,
       0.31169353, 0.59914986, 0.25419271, 0.20749272, 0.45255578,
       0.22534436, 0.52814859, 0.28097162, 0.27900307, 0.29328353,
       0.42606651, 0.29946339, 0.23042639, 0.19413219, 0.22021859,
       0.28035569, 0.19921427, 0.18729705, 0.41816962, 0.21645975,
       0.16137977, 0.19921804, 0.19644288, 0.41959505, 0.26597

In [93]:
Hidden_Dim

100

In [124]:
rnn_model = build_graph(feed_previous=False)

(5, ?, 108)
Uncer_T: (?, 5, 108)
len_enc_inp: 9
[<tf.Tensor 'add_17:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_19:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_21:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_23:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_25:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_27:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_29:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_31:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_33:0' shape=(?, 108) dtype=float32>]
len_target_seq: 1


(5, ?, 108)
Output_UnST: (?, 5, 108)
len_enc_inp: 5
[<tf.Tensor 'strided_slice_99:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_100:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_101:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_102:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_103:0' shape=(?, 108) dtype=float32>]
len_target_seq: 1


Uncert_Result_shape (1, ?, 108)
1 (?, 108) (?, 108)
Pred_shape, Uncert_Up: (?, 108) (?, 108)
####################
(?, 108) (?, 108)
(?, 108) (?, 108)
(?, 108) (?, 108)
####################


# Train_Period

In [133]:
epoch = 2
batch_size = 200
tra = 20000
# n_batch = int(tra/ batch_size)
# # 把batch分成多少个sub batch来计算
# subdivisions = 20
# subdivisions_batch_size = int(np.ceil(batch_size / subdivisions))
# print(epoch, batch_size, n_batch, subdivisions, subdivisions_batch_size)

import datetime
#import evaluation
rnn_model = build_graph(feed_previous=False)
saver = tf.train.Saver()
epoch_loss = []
starttime = datetime.datetime.now()
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    min_error = 100
    for i in range(100): 
        Input_Batch,Input_Ext_B,Matrix_Aff_B,Output_Y_B, Qua_Noise_B, ST_VAR_B = generate_samples(50, corup = False,degree = 0)
        #print(Input_Batch.shape,Input_Ext_B.shape,Matrix_Aff_B.shape,Output_Y_B.shape, Qua_Noise_B.shape, ST_VAR_B.shape)
        feed_dict = {rnn_model['Input_Seq']:Input_Batch, rnn_model['Input_Ext']:Input_Ext_B ,rnn_model['Matrix_Aff']:Matrix_Aff_B,rnn_model['Output_Seq']:Output_Y_B,
                     rnn_model['Data_quality']:Qua_Noise_B, rnn_model['ST_variance']: ST_VAR_B}
        _, loss_t1 = sess.run([rnn_model['train_op'], rnn_model['loss']], feed_dict)
        print(i,loss_t1)
        if loss_t1 < 0 or np.isnan(loss_t1)==True:
            print(loss_t1)
            break
        elif (min_error > loss_t1):
            min_error = loss_t1
            temp_saver = rnn_model['saver']()
            save_path = temp_saver.save(sess, os.path.join('../Model/', 'Uncertainty_Quanti_SIP_var_quality'))
            print(0)
            
        Input_Batch,Input_Ext_B,Matrix_Aff_B,Output_Y_B, Qua_Noise_B, ST_VAR_B = generate_samples(50, corup = True,degree = 1)
        #print(Input_Batch.shape,Input_Ext_B.shape,Matrix_Aff_B.shape,Output_Y_B.shape, Qua_Noise_B.shape, ST_VAR_B.shape)
        feed_dict = {rnn_model['Input_Seq']:Input_Batch, rnn_model['Input_Ext']:Input_Ext_B ,rnn_model['Matrix_Aff']:Matrix_Aff_B,rnn_model['Output_Seq']:Output_Y_B,
                     rnn_model['Data_quality']:Qua_Noise_B, rnn_model['ST_variance']: ST_VAR_B}
        _, loss_t2,learning_rate = sess.run([rnn_model['train_op'], rnn_model['loss'],rnn_model['learning_rate']], feed_dict)
        
        print(i,loss_t2,learning_rate)
        if (loss_t2 < 0) or np.isnan(loss_t2)==True :
            print(loss_t2)
            break
        elif (min_error > loss_t2):
            min_error = loss_t2
            temp_saver = rnn_model['saver']()
            # save_path = temp_saver.save(sess, os.path.join('../Model/', 'Uncertainty_Quanti_SIP_var_quality'))
    
print("Checkpoint saved at: ", save_path)

(5, ?, 108)
Uncer_T: (?, 5, 108)
len_enc_inp: 9
[<tf.Tensor 'add_17:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_19:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_21:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_23:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_25:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_27:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_29:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_31:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_33:0' shape=(?, 108) dtype=float32>]
len_target_seq: 1


(5, ?, 108)
Output_UnST: (?, 5, 108)
len_enc_inp: 5
[<tf.Tensor 'strided_slice_99:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_100:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_101:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_102:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_103:0' shape=(?, 108) dtype=float32>]
len_target_seq: 1


Uncert_Result_shape (1, ?, 108)
1 (?, 108) (?, 108)
Pred_shape, Uncert_Up: (?, 108) (?, 108)
####################
(?, 108) (?, 108)
(?, 108) (?, 108)
(?, 108) (?, 108)
####################
50sample generating fisnished!
0 53.334938
0
50sample generating fisnished!
0 53.818783 0.001
50sample generating fisnished!
1 52.32018
0
50sample generating fisnished!
1 44.999302 0.001
50sample generating fisnished!
2 47.23946
50sample generating fisnished!
2 50.326923 0.001
50sample generating fisnished!
3 51.748764
50sample generating fisnished!
3 49.535686 0.001
50sample generating fisnished!
4 51.44833
50sample generating fisnished!
4 38.386883 0.001
50sample generating fisnished!
5 39.15557
50sample generating fisnished!
5 38.62723 0.00098
50sample generating fisnished!
6 39.761883
50sample generating fisnished!
6 40.95131 0.00098
50sample generating fisnished!
7 42.45737
50sample generating fisnished!
7 43.25052 0.00098
50sample generating fisnished!
8 50.43311
50sample generating fisnished!


75 21.809278
50sample generating fisnished!
75 15.794757 0.00073856936
50sample generating fisnished!
76 20.07815
50sample generating fisnished!
76 14.822629 0.00073856936
50sample generating fisnished!
77 19.243704
50sample generating fisnished!
77 17.034248 0.00073856936
50sample generating fisnished!
78 15.09335
50sample generating fisnished!
78 14.689169 0.00073856936
50sample generating fisnished!
79 16.770218
50sample generating fisnished!
79 13.790397 0.00073856936
50sample generating fisnished!
80 14.99193
50sample generating fisnished!
80 13.235217 0.00072379794
50sample generating fisnished!
81 15.118723
50sample generating fisnished!
81 17.195406 0.00072379794
50sample generating fisnished!
82 14.324046
50sample generating fisnished!
82 15.213349 0.00072379794
50sample generating fisnished!
83 15.923252
50sample generating fisnished!
83 14.444448 0.00072379794
50sample generating fisnished!
84 13.03878
50sample generating fisnished!
84 12.915888 0.00072379794
50sample genera

In [137]:
def generate_samples(batch_size, corup = False,degree = 0):
    tt = np.random.choice(range(14*48,4320), batch_size)
   # tt = np.arange(768,816)
    Input_Batch = []
    Input_Ext_B = []
    Matrix_Aff_B = []
    Output_Y_B = []
    Qua_Noise_B = []
    ST_VAR_B = []
    for i in tt:
        FlowData, Exte, Y, Matrix_Aff, Qua_Noise = Data_OrganSample(i,Corup = corup, Degree = degree)
        Input_Batch.append(FlowData)
        Input_Ext_B.append(Exte)
        Matrix_Aff_B.append(Matrix_Aff)
        Output_Y_B.append(Y)
        Qua_Noise_B.append(Qua_Noise)
        _,_,_, ST_var = ST_Variance(FlowData,Matrix_Aff)
        ST_VAR_B.append(ST_var)
    Input_Batch = np.array(Input_Batch)/100
    Input_Ext_B = np.array(Input_Ext_B)
    Matrix_Aff_B = np.array(Matrix_Aff_B)
    Output_Y_B = np.array(Output_Y_B) /100
    Qua_Noise_B = np.array(np.array(Qua_Noise_B))/100
    Output_Y_B = np.expand_dims(Output_Y_B, axis = 1)
    # 除以10 作为指示
    ST_VAR_B = np.array(ST_VAR_B)/100  #00
    print(str(batch_size)+"sample generating fisnished!")
    return Input_Batch,Input_Ext_B,Matrix_Aff_B,Output_Y_B,Qua_Noise_B,ST_VAR_B

# Test_Period

In [135]:
# 屏蔽tensorflow中的warning
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

rnn_model = build_graph(feed_previous=True)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    Pre_Pred = []
    Up_Pred = []
    Up_Uncert = []
    Output_Label = []
    Output_UnST = []
    Uncer_T = []
    Uncert_Result = []
    sess.run(init)
    saver = rnn_model['saver']().restore(sess, os.path.join('../Model/', 'Uncertainty_Quanti_SIP_var_quality'))
    Input_Batch_T,Input_Ext_B_T,Matrix_Aff_B_T,Output_Y_B_T, Qua_Noise_B_T, ST_VAR_B_T = generate_samples(50)
    feed_dict = {rnn_model['Input_Seq']:Input_Batch_T, rnn_model['Input_Ext']:Input_Ext_B_T ,rnn_model['Matrix_Aff']:Matrix_Aff_B_T,rnn_model['Output_Seq']:Output_Y_B_T,
                     rnn_model['Data_quality']:Qua_Noise_B_T, rnn_model['ST_variance']: ST_VAR_B_T
                }
    Pred, Pred_Up, Uncert_Up,Y_label,Output6,Uncer_T1,Output_Unst,Uncert_res = sess.run([rnn_model['Pred_Result'], rnn_model['Pred_Up'], rnn_model['Uncert_Up'], rnn_model['Output_Seq'],rnn_model['Output6'], rnn_model['Uncer_T'],rnn_model['Output_UnST'],rnn_model['Uncert_Result']], feed_dict)
  #  Pred, Pred_Up, Uncert_Up,Y_label,Output6 = sess.run([rnn_model['Pred_Result'], rnn_model['Pred_Up'], rnn_model['Uncert_Up'], rnn_model['Output_Seq'],rnn_model['Output6']], feed_dict)
    Output_UnST.append(Output_Unst)
    Pre_Pred.append(Pred)
    Up_Pred.append(Pred_Up)
    Up_Uncert.append(Uncert_Up)
    Output_Label.append(Y_label)
    Uncer_T.append(Uncer_T1)
    Uncert_Result.append(Uncert_res)
Pre_Pred = np.array(Pre_Pred)
Up_Pred = np.array(Up_Pred)
Up_Uncert = np.array(Up_Uncert)
Output_Label = np.array(Output_Label)
Uncer_T = np.array(Uncer_T)
Output_UnST = np.array(Output_UnST)
Uncert_Result = np.array(Uncert_Result)


(5, ?, 108)
Uncer_T: (?, 5, 108)
len_enc_inp: 9
[<tf.Tensor 'add_17:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_19:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_21:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_23:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_25:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_27:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_29:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_31:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'add_33:0' shape=(?, 108) dtype=float32>]
len_target_seq: 1


(5, ?, 108)
Output_UnST: (?, 5, 108)
len_enc_inp: 5
[<tf.Tensor 'strided_slice_99:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_100:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_101:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_102:0' shape=(?, 108) dtype=float32>, <tf.Tensor 'strided_slice_103:0' shape=(?, 108) dtype=float32>]
len_target_seq: 1


Uncert_Result_shape (1, ?, 108)
1 (?, 108) (?, 108)
Pred_shape, Uncert_Up: (?, 108) (?, 108)
####################
(?, 108) (?, 108)
(?, 108) (?, 108)
(?, 108) (?, 108)
####################
INFO:tensorflow:Restoring parameters from ../Model/Uncertainty_Quanti_SIP_var_quality


INFO:tensorflow:Restoring parameters from ../Model/Uncertainty_Quanti_SIP_var_quality


50sample generating fisnished!


In [136]:
#-----------Evaluation----------------#
MAPE = 0
RMSE = 0
K = 50
k = 0
for i in  range(K):
    rmse = (np.mean(np.square(Output_Label[0,i,0,:]-abs(Up_Pred[0,i,:]))))
    RMSE = RMSE + rmse
    k = k +1
RMSE = np.sqrt(RMSE/k)
print("RMSE ", RMSE, k)


k = 0
#Pre_Pred = np.where(Pre_Pred<=0, 1, Pre_Pred)
#Output_Label = np.where(Output_Label==0, 1, Output_Label)
for i in  range(K):
    mape = abs(np.mean((Output_Label[0,i,0,:]-abs(Up_Pred[0,i,:]))/Output_Label[0,i,0,:]))
    if mape<1: # 剔除异常点,如label非常接近0的点
        MAPE = MAPE + mape
        k = k +1
MAPE = MAPE/k

print("MAPE,RMSE ", MAPE, RMSE, k)

## Calculate the calibration
count = 0
factor =5
for i in range(K):
    for j in range(Node):
        if (Up_Pred[0,i,j]-factor*Output_Unst[i,-1,j] < Output_Label[0,i,0,j]) & (Output_Label[0,i,0,j] <Up_Pred[0,i,j]+factor*Output_Unst[i,-1,j]):
            count = count + 1
Recalibrate_rate = count/(Node*K)
Recalibrate_rate

RMSE  2.4641870748975383 50
MAPE,RMSE  0.23027673619303932 2.4641870748975383 47


0.42685185185185187